In [24]:
import random
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

df = pd.read_csv('2013-2014.csv')
n = df.last_valid_index()

In [25]:
PARAMETRES = ['FTHG','FTAG','FTR','HTHG','HTAG','HTR','HS','AS','HST','AST','HF','AF','HC','AC','HY','AY','HR','AR','B365H','B365D','B365A','BWH','BWD','BWA','IWH','IWD','IWA','LBH','LBD','LBA','PSH','PSD','PSA','WHH','WHD','WHA','SJH','SJD','SJA','VCH','VCD','VCA','Bb1X2','BbMxH','BbAvH','BbMxD','BbAvD','BbMxA','BbAvA','BbOU','BbMx>2.5','BbAv>2.5','BbMx<2.5','BbAv<2.5','BbAH','BbAHh','BbMxAHH','BbAvAHH','BbMxAHA','BbAvAHA','PSCH','PSCD','PSCA']

In [26]:
ENTREES = ['AwayTeam','HomeTeam']

In [27]:
SORTIES = ['FTR']

In [28]:
len(SORTIES)

1

In [29]:
df.loc[df['FTR'] == 'H',['FTR']] = float(2.0)
df.loc[df['FTR'] == 'D',['FTR']] = float(1.0)
df.loc[df['FTR'] == 'A',['FTR']] = float(0.0)
df.loc[df['HTR'] == 'H',['HTR']] = float(2.0)
df.loc[df['HTR'] == 'D',['HTR']] = float(1.0)
df.loc[df['HTR'] == 'A',['HTR']] = float(0.0)

In [30]:
for param in PARAMETRES :
    df.loc[df[param].isnull(),param] = np.round(df[param].mean())

In [31]:
for i in range(0,n+1):
    l=0
    for r in range (0,len(df['HomeTeam'].iloc[i])):
        l =+ float(ord((df['HomeTeam'].iloc[i])[r])*r)
    df['HomeTeam'].iloc[i] = l

for i in range(0,n+1):
    l=0
    for r in range (0,len(df['AwayTeam'].iloc[i])):
        l =+ float(ord((df['AwayTeam'].iloc[i])[r])*r)
    df['AwayTeam'].iloc[i] = l

C:\Users\tautu\AppData\Local\Temp\ipykernel_14544\3111569354.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['HomeTeam'].iloc[i] = l
C:\Users\tautu\AppData\Local\Temp\ipykernel_14544\3111569354.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['AwayTeam'].iloc[i] = l


In [32]:
df.isna().sum()

Div         0
Date        0
HomeTeam    0
AwayTeam    0
FTHG        0
           ..
BbMxAHA     0
BbAvAHA     0
PSCH        0
PSCD        0
PSCA        0
Length: 67, dtype: int64

In [33]:
X_train = df[ENTREES][:int(n*0.8)]
Y_train = df[SORTIES][:int(n*0.8)].values
X_test = df[ENTREES][int(n*0.8):]
Y_test = df[SORTIES][int(n*0.8):].values

In [34]:
df = df.astype(np.float32)

ValueError: could not convert string to float: 'F1'

In [ ]:
df = df.sample(frac=1, random_state=217)

In [ ]:
def entropy(p):
    if p == 0:
        return 0
    elif p == 1:
        return 0
    else:
        return (- (p * np.log2(p) + (1 - p) * np.log2(1-p)))

def information_gain(left_child, right_child):
    parent = left_child + right_child
    p_parent = parent.count(1) / len(parent) if len(parent) > 0 else 0
    p_left = left_child.count(1) / len(left_child) if len(left_child) > 0 else 0
    p_right = right_child.count(1) / len(right_child) if len(right_child) > 0 else 0
    IG_p = entropy(p_parent)
    IG_l = entropy(p_left)
    IG_r = entropy(p_right)
    return IG_p - (len(left_child) / len(parent) * IG_l) - (len(right_child) / len(parent) * IG_r)

In [ ]:
def predict_tree(tree, X_test):
    feature_idx = tree['feature_idx']

    if X_test[feature_idx] <= tree['split_point']:
        if type(tree['left_split']) == dict:
            return predict_tree(tree['left_split'], X_test)
        else:
            value = tree['left_split']
            return value
    else:
        if type(tree['right_split']) == dict:
            return predict_tree(tree['right_split'], X_test)
        else:
            return tree['right_split']

In [ ]:
def draw_bootstrap(X_train, Y_train):
    bootstrap_indices = list(np.random.choice(range(len(X_train)), len(X_train), replace = True))
    oob_indices = [i for i in range(len(X_train)) if i not in bootstrap_indices]
    X_bootstrap = X_train.iloc[bootstrap_indices].values
    Y_bootstrap = Y_train.iloc[bootstrap_indices]
    X_oob = X_train.iloc[oob_indices].values
    Y_oob = Y_train.iloc[oob_indices]
    return X_bootstrap, Y_bootstrap, X_oob, Y_oob

def oob_score(tree, X_test, Y_test):
    mis_label = 0
    for i in range(len(X_test)):
        pred = predict_tree(tree, X_test[i])
        if pred != Y_test[i]:
            mis_label += 1
    return mis_label / len(X_test)

In [ ]:
def find_split_point(X_bootstrap, Y_bootstrap, max_features):
    feature_ls = list()
    num_features = len(X_bootstrap[0])

    while len(feature_ls) <= max_features:
        feature_idx = random.sample(range(num_features), 1)
        if feature_idx not in feature_ls:
            feature_ls.extend(feature_idx)

    best_info_gain = -999
    node = None
    for feature_idx in feature_ls:
        for split_point in X_bootstrap[:,feature_idx]:
            left_child = {'X_bootstrap': [], 'Y_bootstrap': []}
            right_child = {'X_bootstrap': [], 'Y_bootstrap': []}

            # split children for continuous variables
            if type(split_point) in [int, float]:
                for i, value in enumerate(X_bootstrap[:,feature_idx]):
                    if value <= split_point:
                        left_child['X_bootstrap'].append(X_bootstrap[i])
                        left_child['Y_bootstrap'].append(Y_bootstrap[i])
                    else:
                        right_child['X_bootstrap'].append(X_bootstrap[i])
                        right_child['Y_bootstrap'].append(Y_bootstrap[i])
            # split children for categoric variables
            else:
                for i, value in enumerate(X_bootstrap[:,feature_idx]):
                    if value == split_point:
                        left_child['X_bootstrap'].append(X_bootstrap[i])
                        left_child['Y_bootstrap'].append(Y_bootstrap[i])
                    else:
                        right_child['X_bootstrap'].append(X_bootstrap[i])
                        right_child['Y_bootstrap'].append(Y_bootstrap[i])

            split_info_gain = information_gain(left_child['Y_bootstrap'], right_child['Y_bootstrap'])
            if split_info_gain > best_info_gain:
                best_info_gain = split_info_gain
                left_child['X_bootstrap'] = np.array(left_child['X_bootstrap'])
                right_child['X_bootstrap'] = np.array(right_child['X_bootstrap'])
                node = {'information_gain': split_info_gain,
                        'left_child': left_child,
                        'right_child': right_child,
                        'split_point': split_point,
                        'feature_idx': feature_idx}
    return node

In [ ]:
def terminal_node(node):
    Y_bootstrap = node['Y_bootstrap']
    pred = max(Y_bootstrap, key = Y_bootstrap.count)
    return pred

In [ ]:
def split_node(node, max_features, min_samples_split, max_depth, depth):
    left_child = node['left_child']
    right_child = node['right_child']

    del(node['left_child'])
    del(node['right_child'])

    if len(left_child['Y_bootstrap']) == 0 or len(right_child['Y_bootstrap']) == 0:
        empty_child = {'Y_bootstrap': left_child['Y_bootstrap'] + right_child['Y_bootstrap']}
        node['left_split'] = terminal_node(empty_child)
        node['right_split'] = terminal_node(empty_child)
        return node

    if depth >= max_depth:
        node['left_split'] = terminal_node(left_child)
        node['right_split'] = terminal_node(right_child)
        return node

    if len(left_child['X_bootstrap']) <= min_samples_split:
        node['left_split'] = node['right_split'] = terminal_node(left_child)
    else:
        node['left_split'] = find_split_point(left_child['X_bootstrap'], left_child['Y_bootstrap'], max_features)
        split_node(node['left_split'], max_depth, min_samples_split, max_depth, depth + 1)
    if len(right_child['X_bootstrap']) <= min_samples_split:
        node['right_split'] = node['left_split'] = terminal_node(right_child)
    else:
        node['right_split'] = find_split_point(right_child['X_bootstrap'], right_child['Y_bootstrap'], max_features)
        split_node(node['right_split'], max_features, min_samples_split, max_depth, depth + 1)

In [ ]:
def build_tree(X_bootstrap, Y_bootstrap, max_depth, min_samples_split, max_features):
    root_node = find_split_point(X_bootstrap, Y_bootstrap, max_features)
    split_node(root_node, max_features, min_samples_split, max_depth, 1)
    return root_node

def random_forest(X_train, Y_train, n_estimators, max_features, max_depth, min_samples_split):
    tree_ls = list()
    oob_ls = list()
    for i in range(n_estimators):
        X_bootstrap, Y_bootstrap, X_oob, Y_oob = draw_bootstrap(X_train, Y_train)
        tree = build_tree(X_bootstrap, Y_bootstrap, max_features, max_depth, min_samples_split)
        tree_ls.append(tree)
        oob_error = oob_score(tree, X_oob, Y_oob)
        oob_ls.append(oob_error)
    print("OOB estimate: {:.2f}".format(np.mean(oob_ls)))
    return tree_ls

In [ ]:
def predict_rf(tree_ls, X_test):
    pred_ls = list()
    for i in range(len(X_test)):
        ensemble_preds = [predict_tree(tree, X_test.values[i]) for tree in tree_ls]
        final_pred = max(ensemble_preds, key = ensemble_preds.count)
        pred_ls.append(final_pred)
    return np.array(pred_ls)

In [ ]:
n_estimators = 100
max_features = 3
max_depth = 10
min_samples_split = 2

In [ ]:
model = random_forest(X_train, Y_train, n_estimators=100, max_features=3, max_depth=10, min_samples_split=2)

AttributeError: 'numpy.ndarray' object has no attribute 'iloc'

In [ ]:
acc=0
for i in range (0,len(Y_test)):
    if (results[i] == Y_test['FTR'].iloc[i]) :
        acc =+ 1
print(acc/len(Y_test))

0.012987012987012988
